In [ ]:
# Exercise 1 #

import ipaddress

def analyse_ip(ip_str, cidr_prefix):
    # Create an IP interface object
    ip_with_cidr = f"{ip_str}/{cidr_prefix}"
    ip = ipaddress.ip_interface(ip_with_cidr)
    network = ip.network

    MASK = '255.255.0.0'

    # host = ipaddress.IPv4Address(ip)
    net = ipaddress.IPv4Network(ip_str + '/' + MASK, False)

    print(f"Address: {ip.ip}")
    print(f"Network: {ip.network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Is private: {ip.ip.is_private}")
    print(f"Is global: {ip.ip.is_global}")
    print(f"Broadcast address: {ip.network.broadcast_address}")
    print(f"CIDR Notation: {network.with_prefixlen}")
    #print(f"First and last usable host addresses: {n[0]}, {n[-1]}")

    # First and last usable host addresses
    all_hosts = list(network.hosts())
    if all_hosts:
        first_host = all_hosts[0]
        last_host = all_hosts[-1]
    else:
        first_host = last_host = "N/A"  # No usable hosts in /31 or /32 networks
    
    print(f"First usable host: {first_host}")
    print(f"Last usable host: {last_host}")
    print(f"Number of usable hosts: {len(all_hosts)}")

    # List all hosts in the network
    if ip.network.num_addresses < 256: # Only for small networks
        print("\nHosts in network:")
        for host in ip.network.hosts():
            print(host)

# Example usage
analyse_ip('192.168.1.1', 24)

Address: 192.168.1.1
Network: 192.168.1.0/24
Netmask: 255.255.255.0
Is private: True
Is global: False
Broadcast address: 192.168.1.255
CIDR Notation: 192.168.1.0/24
First usable host: 192.168.1.1
Last usable host: 192.168.1.254
Number of usable hosts: 254


In [7]:
# Exercise 2 #

import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)

print("Your Computer Name is:" + hostname)
print("Your Computer IP Address is:" + IPAddr)

analyse_ip(IPAddr, 24)

Your Computer Name is:AbrarPC
Your Computer IP Address is:10.100.56.130
Address: 10.100.56.130
Network: 10.100.56.0/24
Netmask: 255.255.255.0
Is private: True
Is global: False
Broadcast address: 10.100.56.255
CIDR Notation: 10.100.56.0/24
First usable host: 10.100.56.1
Last usable host: 10.100.56.254
Number of usable hosts: 254


In [10]:
# Exercise 3 #

import socket

uni_ip = socket.gethostbyname('www.gold.ac.uk')

analyse_ip(uni_ip, 24)


Address: 159.100.136.66
Network: 159.100.136.0/24
Netmask: 255.255.255.0
Is private: False
Is global: True
Broadcast address: 159.100.136.255
CIDR Notation: 159.100.136.0/24
First usable host: 159.100.136.1
Last usable host: 159.100.136.254
Number of usable hosts: 254
